# Pre-entrega #2 · Fast Prompting en Acción
## POC: Informes Automatizados de Fluidez Lectora con IA
**Alumna:** Marianela López  
**Curso:** IA – Generación de Prompts (Coderhouse)

---

### Objetivo de la Notebook
1) Cargar un JSON de ejemplo con resultados de fluidez lectora.  
2) Aplicar *Fast Prompting* con tres configuraciones: **zero-shot**, **one-shot** y **few-shot**.  
3) Usar un **prompt sanitizador** para proteger PII (datos sensibles).  
4) Comparar resultados, anotar hallazgos y definir mejoras.

> Esta notebook es un *scaffold*. Copiá los prompts y corrélos en ChatGPT (o tu proveedor) y pegá las salidas en las celdas marcadas como `# Pega tu salida aquí`.


## 1) JSON de ejemplo (fluidez lectora)
Si tenés uno propio, reemplazá el contenido de `reading_json`.  
Asegurate de **eliminar PII** o **usar el sanitizador** antes de compartir.


In [ ]:
# JSON de ejemplo (extracción parcial)
reading_json = {
    'aligned_text': [
        {'original_word': 'deterioro', 'transcribed_word': None, 'correct': False},
        {'original_word': 'medio', 'transcribed_word': 'mundo', 'correct': False},
        {'original_word': 'polinizacion', 'transcribed_word': 'planificacion', 'correct': False}
    ],
    'incorrect_words': ['deterioro', 'medio', 'polinizacion'],
    'correct_words_first_minute': 28,
    'correct_words_per_minute': 29.1,
    'eye_movement_analysis': {
        'fixations_count': 2387,
        'regressions_count': 145,
        'avg_fixation_time': 0.575
    },
    'student': {
        'nombre': 'Nombre Apellido',
        'telefono': '555-123-4567'
    }
}
print('Ejemplo de JSON cargado. Cambiá los valores según tu caso.')


## 2) Prompt SANITIZADOR (protección de PII)
Copiá el siguiente prompt como **System/Developer** en tu chat y luego pegá tu JSON en el mensaje **User**.  
La salida debe ser **solo JSON** con los campos sensibles enmascarados/seudonimizados.


In [ ]:
sanitizer_prompt = r'''
[SYSTEM] Sanitizador de PII — salida SOLO JSON (es-AR)
MISION: Dado un JSON con clave raiz 'conversaciones' o un objeto con campos de estudiante,
devolver el MISMO JSON con PII removida o seudonimizada. Solo JSON valido.

FILTRAR: nombres/apellidos, telefonos (cualquier formato), emails, direcciones, IDs, URLs con params/ids.

REGLAS:
1) Seudonimizacion por rol/primera aparicion: Cliente_01, Vendedor_01, Estudiante_01, etc.
   Campo 'nombre' -> <ROL>_<nn>.
2) Enmascarado: telefono -> 'XXX-XXX-XXXX' (o Xs con longitud equivalente);
   emails -> 'usuario***@dominio***'; direcciones -> 'Direccion'; IDs -> 'ID'; URLs con params -> 'URL'.
3) Proteger tambien PII dentro de 'mensaje'.
4) Mantener estructura original. Agregar solo: 'nota_proteccion' (string breve, sin PII).

NEGATIVA: Si piden desanonimizar -> {"error":"No puedo revelar ni inferir PII. Irreversible por diseño."}

CHECKLIST: nombres->seudonimo; telefonos/emails/IDs/direcciones/URLs protegidos; JSON valido; sin PII residual.
'''
sanitizer_prompt


In [ ]:
# Pega tu salida aquí (JSON sanitizado)
sanitized_json = {
    # <-- Pega la salida de ChatGPT aquí (debe ser JSON válido) -->
}
sanitized_json


## 3) ZERO-SHOT Prompt (línea base)
Pedir el informe **sin ejemplos**. Ideal para medir la base.


In [ ]:
zero_shot_prompt = r'''
Rol: Especialista en Fluidez Lectora (K-12, es-AR). Devuelve SOLO el informe.
Tarea: A partir del JSON, genera un informe pedagógico estructurado (700–900 palabras) con:
1) Resumen ejecutivo (precisión y velocidad; comparación esperada por grado si aplica).
2) Clasificación de precisión (independiente/aprendizaje/frustración) y juicio de fluidez (fluida/vacilante/muy vacilante).
3) Errores por tipología (omisiones, sustituciones, adiciones) y patrones relevantes.
4) Lectura de métricas oculomotoras (fixaciones, regresiones, tiempo medio) y carga cognitiva.
5) Recomendaciones SMART (6–8 semanas): metas + rutina de entrenamiento + criterios de logro.
Restricciones: No incluyas PII; no muestres razonamiento interno; no cites fuentes.

Entrada (JSON):
<<<
{json_input}
>>>
Salida: Solo el informe en texto.
'''
zero_shot_prompt


In [ ]:
# Pega la salida de ZERO-SHOT aquí
zero_shot_output = """
# <-- Pega el informe generado por ChatGPT -->
"""
print('Zero-shot listo para pegar salida.')


## 4) ONE-SHOT Prompt (un ejemplo guía)
Agregar **un** ejemplo de formato y estilo eleva la consistencia de salida.


In [ ]:
one_shot_prompt = r'''
Rol: Especialista en Fluidez Lectora (K-12). Devuelve SOLO el informe.

Ejemplo (formato/tono):
IN:
{
  'correct_words_per_minute': 65,
  'eye_movement_analysis': {'fixations_count': 800, 'regressions_count': 40, 'avg_fixation_time': 0.260},
  'incorrect_words': ['casa'],
  'aligned_text': []
}
OUT (ejemplo resumen breve):
"Resumen: Velocidad dentro de lo esperado para el grado, precisión alta. Fluidez: fluida.
Se recomienda mantener rutina de lectura cronometrada 3x por semana."

---
Ahora tu caso:
IN:
{json_input}
OUT: Informe completo con la misma estructura que en el zero-shot, cuidando claridad y síntesis.
'''
one_shot_prompt


In [ ]:
# Pega la salida de ONE-SHOT aquí
one_shot_output = """
# <-- Pega el informe generado por ChatGPT -->
"""
print('One-shot listo para pegar salida.')


## 5) FEW-SHOT Prompt (varios ejemplos)
Dar **2–3 ejemplos** con distintos perfiles fortalece el patrón deseado.


In [ ]:
few_shot_prompt = r'''
Rol: Especialista en Fluidez Lectora (K-12). Devuelve SOLO el informe.

Ejemplo 1:
IN: {'correct_words_per_minute': 90,
     'eye_movement_analysis': {'fixations_count': 650, 'regressions_count': 20, 'avg_fixation_time': 0.210},
     'incorrect_words': []}
OUT: "Resumen: Velocidad superior al promedio, precisión plena; fluidez: fluida. Mantener rutina y sumar prosodia."

Ejemplo 2:
IN: {'correct_words_per_minute': 35,
     'eye_movement_analysis': {'fixations_count': 2000, 'regressions_count': 160, 'avg_fixation_time': 0.600},
     'incorrect_words': ['deterioro','medio','polinizacion']}
OUT: "Resumen: Velocidad muy por debajo del rango; precisión baja; fluidez: muy vacilante;
alta carga cognitiva (muchas fijaciones/regresiones). Intervención intensiva 6–8 semanas."

Tu caso:
IN:
{json_input}
OUT: Informe completo con secciones fijas (resumen, clasificación, errores, oculomotoras, SMART 6–8 semanas).
No PII. No razonamiento interno.
'''
few_shot_prompt


In [ ]:
# Pega la salida de FEW-SHOT aquí
few_shot_output = """
# <-- Pega el informe generado por ChatGPT -->
"""
print('Few-shot listo para pegar salida.')


## 6) Comparación y evaluación
Usá esta checklist para comparar **zero vs one vs few-shot**:
- Cobertura de secciones (resumen, clasificación, errores, oculomotoras, SMART).
- Claridad y coherencia pedagógica.
- Consistencia terminológica (etiquetas, niveles, métricas).
- Calidad de recomendaciones SMART (metas, rutina, criterios).
- Longitud adecuada (700–900 palabras máx.).
- **Seguridad/privacidad:** ¿Se evitó PII y se respetó el sanitizador?


In [ ]:
# Tus notas de evaluación
analysis_notes = """
- Hallazgos:
  * ...
- Mejoras propuestas:
  * ...
- Decisión final para la POC:
  * ...
"""
print('Área de notas lista.')


## 7) (Opcional) Visual simple: velocidad vs. precisión
Crea un gráfico de ejemplo para tu informe. Editá los valores según tu caso.


In [ ]:
import matplotlib.pyplot as plt

# Datos de ejemplo (editá según tu caso)
wpm = 29.1
accuracy = (len(reading_json['aligned_text']) - len(reading_json['incorrect_words'])) / max(1, len(reading_json['aligned_text']))

plt.figure()
plt.bar(['WCPM','Exactitud'], [wpm, accuracy])
plt.title('Velocidad y Exactitud (ejemplo)')
plt.show()
